 # Worm Analysis
 This code is written by Gabriel Nützi, gnuetzi (at) gmail (døt) com.    
 Its released under GPL 3.0 License
 
 It was written for biological engineering problems where worms needed to be analyzed from a set of pictures.

In [179]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

%matplotlib tk 
# or Qt or GTK or what ever GUI backend you have

## Define Polygon Outline Points Graphically

In [198]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlim(0,3)
ax.set_ylim(0,3)
points = []

def onclick(event):
    print('add point x=%f, y=%f' % (event.xdata, event.ydata))
    p = [event.xdata, event.ydata]
    ax.plot([event.xdata],[event.ydata],"bo")
    fig.canvas.draw()
    points.append(p)

cid = fig.canvas.mpl_connect('button_press_event', onclick)

def onclose(event):
    p = np.array(points)
    np.save("OutlinePoints-3",np.array(points))
    print("Saved polygon ouline points")

fig.canvas.mpl_connect('close_event', onclose)

plt.show()

add point x=0.203713, y=2.271807
add point x=0.193796, y=2.199116
add point x=0.190491, y=2.092218
add point x=0.193796, y=1.946836
add point x=0.193796, y=1.835661
add point x=0.193796, y=1.707383
add point x=0.193796, y=1.570553
add point x=0.200408, y=1.489310
add point x=0.180574, y=1.361032
add point x=0.190491, y=1.279789
add point x=0.200408, y=1.121579
add point x=0.203713, y=0.980473
add point x=0.207019, y=0.886403
add point x=0.210325, y=0.770952
add point x=0.213630, y=0.689709
add point x=0.226853, y=0.625570
add point x=0.243382, y=0.565707
add point x=0.259910, y=0.505844
add point x=0.286356, y=0.424601
add point x=0.329330, y=0.403221
add point x=0.332635, y=0.403221
add point x=0.349164, y=0.407497
add point x=0.359081, y=0.437429
add point x=0.402055, y=0.505844
add point x=0.402055, y=0.522948
add point x=0.405361, y=0.612742
add point x=0.411972, y=0.817987
add point x=0.421889, y=0.963369
add point x=0.438418, y=1.117303
add point x=0.448335, y=1.254133
add point 

## Load Voronoi points from the saved file
Run all cells below for certain file number (Cell -> Run All Below)

In [199]:
fileNr = 3

points = np.load("OutlinePoints-%i.npy" % fileNr) 
# points = [ [p1.x,p1.y], 
#            [p2.x,p2.y] ]
#    
pointsClosed=np.vstack([ points, [points[0]] ])

## Voronoi Diagram

In [200]:
from scipy.spatial import Voronoi, voronoi_plot_2d

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111)
ax.set_title("Voronoi Diagram and Extraction of Centerline")
plt.show()

vor = Voronoi(points)

# polygon points (blue)
ax.plot(pointsClosed[:,0],pointsClosed[:,1],"b-",lw=5)

#ridge points
for p in vor.ridge_points:
    l = np.stack([vor.points[p[0]], vor.points[p[1]]])
    ax.plot(l[:,0],l[:,1],"r-")

fig=voronoi_plot_2d(vor,ax=ax)
ax.set_aspect('equal', 'datalim')

# red points are vor.vertices

## Extract Median Line

In [201]:
# filter all vertices which are not inside polygon
import matplotlib.path as mplPath
    
path=mplPath.Path(points)
contains=path.contains_points(vor.vertices)
vor.verticesInside = set(np.where(contains==True)[0].tolist()) # make set (because faster value lookup)

# filter all ridge_vertices which do not contain indices of vorVerticesInside
vor.validRidges = filter( lambda r: r[0] in vor.verticesInside and r[1] in vor.verticesInside , 
                          vor.ridge_vertices )


# plot valid ridges (black)
for r in vor.validRidges:
    l = np.stack([vor.vertices[r[0]], vor.vertices[r[1]]])
    ax.plot(l[:,0],l[:,1],"k-", lw=5)

fig.canvas.draw()

fig.savefig("OutlinePoints-%i.pdf" % fileNr)

## Try Skeletonization

In [202]:
from skimage import *
import skimage.io
import skimage.morphology as morph

In [203]:
def loadImage(f):
    return skimage.img_as_float(skimage.io.imread(f))

img=loadImage("../worm-f-024.tif");
plt.figure(figsize=(12,8))
plt.imshow(img, cmap=cm.gray)

In [204]:
skeleton = morph.skeletonize(img)

In [205]:
fig=plt.figure(figsize=(12,8))
plt.imshow(skeleton, cmap=cm.gray)
fig.savefig("Skeleton.pdf")